In [18]:
import pandas as pd
import numpy as np

# 지리적 거리 계산
from geopy.distance import geodesic 

# **1. 데이터 로드**

## 1) 아파트 데이터
- 지하철역, 병원 수까지 추가된 데이터

In [21]:
house = pd.read_csv('apartment3.csv')
# house.drop(['Unnamed: 0'], axis=1, inplace=True)

print('전체 shape : ', house.shape)
house.head(1)

전체 shape :  (3142, 29)


,아파트명,법정동주소,위도,경도,세대수,임대세대수,최고층,최저층,최대공급면적,최소공급면적,...,현관구조,입주예정일,공급액(만원),대형건설사,location,지하철역,지하철역_거리(km),1차병원,2차병원,3차병원
0,올림픽파크 포레온,서울특별시 강동구 둔촌1동 170-1,37.522886,127.140539,12032,1046.0,35.0,20.0,114.42,46.21,...,혼합식,202501,"132,040",YES,"(37.5228859, 127.140539)",9호선 둔촌오륜역,0.6223,46,16,1


## 2) 대학교 데이터

In [46]:
university = pd.read_excel('학교개황(20220310기준).xlsx')
print('학교 데이터 수정 전 shape : ', university.shape)
# university.head(1)
# university.info()

학교 데이터 수정 전 shape :  (2013, 21)


In [47]:
university.columns

Index(['학교구분', '학교코드', '학교명', '본분교', '학제', '지역', '설립구분', '관련법령', '법인명', '학교상태',
       '중문학교명', '영문학교명', '주소', '영문주소', '중문주소', '우편번호', '학교개교일', '학교홈페이지',
       '총장(대학원장)명', '학교대표\n번호', '학교대표\n팩스번호'],
      dtype='object')

#### 데이터 확인

In [37]:
# <대학교 데이터 필요 컬럼만 가져와 확인>

university = university[['학교명', '학교구분', '본분교', '지역', '설립구분', '우편번호', '주소', '법인명', '학교상태', '학교개교일']]

print('1. 학교 상태 유니크 값 : ', university['학교상태'].unique()) # -> 폐교인 행 drop하기
print('2. 본분교 유니크 값 : ', university['본분교'].unique())
print('3. 설립구분 유니크 값 : ', university['설립구분'].unique()) 
print('4. 지역 유니크 값 : ', university['지역'].unique()) # -> 서울, 경기, 인천 제외한 행 모두 drop하기

1. 학교 상태 유니크 값 :  ['기존' '폐교' '신설']
2. 본분교 유니크 값 :  ['본교' '제2캠퍼스' '제3캠퍼스' '분교' '제4캠퍼스']
3. 설립구분 유니크 값 :  ['국립' '국립대법인' '공립' '사립' '특별법법인' '특별법국립' '기타']
4. 지역 유니크 값 :  ['강원' '대구' '경남' '충남' '전북' '경북' '전남' '부산' '서울' '광주' '제주' '대전' '충북' '경기'
 '세종' '인천' '울산']


In [38]:
# <원본 데이터의 서울, 경기, 인천 학교수>

seoul_schools = university[university['지역'].isin(['서울'])]
gyeonggi_schools = university[university['지역'].isin(['경기'])]
incheon_schools = university[university['지역'].isin(['인천'])]


print('서울의 학교는 몇개? : ', len(seoul_schools['학교명']))
print('경기의 학교는 몇개? : ', len(gyeonggi_schools['학교명']))
print('인천의 학교는 몇개? : ', len(incheon_schools['학교명']),'\n')

print('서울의 학교는 몇개?(유니크 값) : ', len(seoul_schools['학교명'].unique())) # -2개 
print('경기의 학교는 몇개?(유니크 값) : ', len(gyeonggi_schools['학교명'].unique())) # -2개
print('인천의 학교는 몇개?(유니크 값) : ', len(incheon_schools['학교명'].unique()), '\n') # -1개

# 지역이 서울, 경기, 인천 지역인 학교의 수
total = seoul_schools+gyeonggi_schools+incheon_schools
print('지역이 서울, 경기, 인천 지역인 학교의 수 :', len(total))

서울의 학교는 몇개? :  570
경기의 학교는 몇개? :  335
인천의 학교는 몇개? :  43 

서울의 학교는 몇개?(유니크 값) :  568
경기의 학교는 몇개?(유니크 값) :  333
인천의 학교는 몇개?(유니크 값) :  42 

지역이 서울, 경기, 인천 지역인 학교의 수 : 948


# **2. 대학교 데이터 전처리**

## 1) 학교가 폐교인 것 drop
- '학교상태' 열에서 '폐교'인 행을 삭제

In [55]:
# 위도 경도 포함된 university 데이터 불러오기 -> 구글 시트에서 위도 경도 계산
university = pd.read_csv('university.csv')
university.head(1)

,학교명,학교구분,본분교,지역,설립구분,우편번호,주소,Latitude,Longitude,법인명,학교상태,학교개교일
0,서울대학교,대학,본교,서울,국립대법인,8826,"서울특별시 관악구 관악로 1 (신림동, 서울대학교)",37.480967,126.953148,서울대학교 법인,기존,1946-08-22


In [57]:
# 학교상태가 폐교인 것 
closed_schools = university[university['학교상태'] == '폐교']
print('학교 상태가 폐교인 것 몇 행? :', closed_schools.shape[0])

# 전체 2013행 중 435행(폐교인 것) 제외하기
university.drop(university[university['학교상태'] == '폐교'].index, axis=0, inplace=True)

# 학교상태 폐교 삭제 & 컬럼명 일부만
print('학교 데이터 수정 후 shape : ', university.shape)
print('학교 상태 유니크 값 : ', university['학교상태'].unique())

학교 상태가 폐교인 것 몇 행? : 0
학교 데이터 수정 후 shape :  (753, 12)
학교 상태 유니크 값 :  ['기존' '신설']


In [40]:
# 폐교 행 drop 후 서울, 경기, 인천 학교 수
seoul_schools = university[university['지역'].isin(['서울'])]
gyeonggi_schools = university[university['지역'].isin(['경기'])]
incheon_schools = university[university['지역'].isin(['인천'])]

# 지역이 서울, 경기, 인천 지역인 학교의 수
total = seoul_schools+gyeonggi_schools+incheon_schools
print('폐교 drop 후 지역이 서울, 경기, 인천 지역인 학교의 수 :', len(total))

폐교 drop 후 지역이 서울, 경기, 인천 지역인 학교의 수 : 753


## 2) 지역 필터링
- '서울', '경기', '인천' 이외의 행을 모두 삭제

In [58]:
# 서울, 경기, 인천 지역이 아닌 행 삭제 
university.drop(university[~university['지역'].isin(['서울', '경기', '인천'])].index, inplace=True)
print('지역이 [서울, 경기, 인천]인 것만 :', university.shape)

지역이 [서울, 경기, 인천]인 것만 : (753, 12)


## 3) 학교구분 필터링
- 대학과 전문대학만 남기고 다은 유형은 삭제

In [59]:
# (수정 전)
university['학교구분'].unique()
university['학교구분'].value_counts()

대학원      564
대학       100
전문대학      51
대학원대학     38
Name: 학교구분, dtype: int64

In [60]:
# (수정 후) 학교 구분이 '대학', '전문대학' 인 경우만
university_df = university[university['학교구분'].isin(['대학', '전문대학'])]
print('학교 구분 확인 :', university_df['학교구분'].value_counts())

학교 구분 확인 : 대학      100
전문대학     51
Name: 학교구분, dtype: int64


## 4) 컬럼명 변경 및 특정 컬럼 삭제

In [62]:
# 컬럼명 변경
new_column_names = {'학교명' : '대학명',
                    '지역' : '대학 지역',
                    '주소' : '대학 주소',
                    'Latitude' : '대학 위도',
                    'Longitude' : '대학 경도'}
university_df.rename(columns=new_column_names, inplace=True)


# 특정 컬럼 삭제
university_df.drop(columns=['우편번호', '법인명', '학교상태', '학교개교일'], inplace=True)

print('컬럼 삭제 후 대학교/대학 전체 shape : ', university_df.shape)
university_df.head(1)

컬럼 삭제 후 대학교/대학 전체 shape :  (151, 8)


/var/folders/ht/vmxckhk151s1x5lpr_f5drm40000gn/T/ipykernel_2261/3143594320.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  university_df.rename(columns=new_column_names, inplace=True)
/var/folders/ht/vmxckhk151s1x5lpr_f5drm40000gn/T/ipykernel_2261/3143594320.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  university_df.drop(columns=['우편번호', '법인명', '학교상태', '학교개교일'], inplace=True)


,대학명,학교구분,본분교,대학 지역,설립구분,대학 주소,대학 위도,대학 경도
0,서울대학교,대학,본교,서울,국립대법인,"서울특별시 관악구 관악로 1 (신림동, 서울대학교)",37.480967,126.953148


# **3. 아파트와 가장 가까운 대학교 개수 계산**
- geodesic를 사용하여 지리적 거리 계산 -> (단점) 너무 오래걸림 
- 아파트에서 1km 반경 내 대학교 수를 계산하여 '대학교_수' 컬럼을 추가

In [63]:
# 반경 내 대학교 수 계산하여 새로운 컬럼 추가 -> (단점)geodesic 너무 오래걸림

km = 1  # 반경 1km
house['대학교_수'] = 0  # 초기화


for h_idx, h_row in house.iterrows() :
    count = 0

     # 대학교 데이터프레임을 반복하여 각 대학교와의 거리를 계산
    for u_idx, u_row in university_df.iterrows() :

         # geodesic 함수를 사용하여 주택 위치와 대학교 위치 간의 거리를 계산
        distance = geodesic((h_row['위도'], h_row['경도']), (u_row['대학 위도'], u_row['대학 경도'])).kilometers

         # 계산된 거리가 '1km' 내에 있는 경우, count 1씩 증가
        if distance <= km :
            count += 1

    house.at[h_idx, '대학교_수'] = count

In [67]:
house.head(1)
house['대학교_수'].value_counts()

0    2635
1     441
2      46
5      13
4       7
Name: 대학교_수, dtype: int64

# **4. csv로 저장**

In [22]:
house.to_csv('apartment4.csv')